In [1]:
from SyMBac.phase_contrast_drawing import run_simulation, get_trench_segments, draw_scene
from SyMBac.general_drawing import generate_curve_props, gen_cell_props_for_draw, get_space_size, convolve_rescale


/home/georgeos/miniconda3/envs/symbac/lib/python3.9/site-packages/SyMBac/general_drawing.py:18: UserWarning: Could not load CuPy for SyMBac, are you using a GPU? Defaulting to CPU convolution.
  warnings.warn("Could not load CuPy for SyMBac, are you using a GPU? Defaulting to CPU convolution.")


In [2]:
resize_amount = 3
pix_mic_conv = 0.0655 ##0.108379937 micron/pix for 60x, 0.0655 for 100x
scale = pix_mic_conv / resize_amount 
sim_length = 100

In [3]:
cell_timeseries, space = run_simulation(
    trench_length=15, 
    trench_width=1.5, 
    cell_max_length=6, #6, long cells # 1.65 short cells
    cell_width= 1, #1 long cells # 0.95 short cells
    sim_length = sim_length,
    pix_mic_conv = pix_mic_conv,
    gravity=0,
    phys_iters=20,
    max_length_var = 3,
    width_var = 0.3,
    save_dir="/tmp/"
) # growth phase

In [4]:
main_segments = get_trench_segments(space)
ID_props = generate_curve_props(cell_timeseries)

In [5]:
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

cell_timeseries_properties = Parallel(n_jobs=-1)(
    delayed(gen_cell_props_for_draw)(a, ID_props) for a in tqdm(cell_timeseries, desc='Timeseries Properties'))

do_transformation = True
offset = 30
label_masks = True
space_size = get_space_size(cell_timeseries_properties)
scenes = Parallel(n_jobs=13)(delayed(draw_scene)(
    cell_properties, do_transformation, space_size, offset, label_masks) for cell_properties in tqdm(cell_timeseries_properties, desc='Scene Draw:'))

Timeseries Properties:   0%|          | 0/98 [00:00<?, ?it/s]

/home/georgeos/miniconda3/envs/symbac/lib/python3.9/site-packages/SyMBac/general_drawing.py:18: UserWarning: Could not load CuPy for SyMBac, are you using a GPU? Defaulting to CPU convolution.
  warnings.warn("Could not load CuPy for SyMBac, are you using a GPU? Defaulting to CPU convolution.")
/home/georgeos/miniconda3/envs/symbac/lib/python3.9/site-packages/SyMBac/general_drawing.py:18: UserWarning: Could not load CuPy for SyMBac, are you using a GPU? Defaulting to CPU convolution.
  warnings.warn("Could not load CuPy for SyMBac, are you using a GPU? Defaulting to CPU convolution.")
/home/georgeos/miniconda3/envs/symbac/lib/python3.9/site-packages/SyMBac/general_drawing.py:18: UserWarning: Could not load CuPy for SyMBac, are you using a GPU? Defaulting to CPU convolution.
  warnings.warn("Could not load CuPy for SyMBac, are you using a GPU? Defaulting to CPU convolution.")
/home/georgeos/miniconda3/envs/symbac/lib/python3.9/site-packages/SyMBac/general_drawing.py:18: UserWarning: Cou

Scene Draw::   0%|          | 0/98 [00:00<?, ?it/s]

In [7]:
cell_timeseries_properties

[[[167.47720085347896,
   45.80152671755725,
   2.937574483607264,
   array([62.61326033, 87.26931858]),
   0.9406013141740135,
   1.085944139308007,
   -0.5204327621930127,
   20]],
 [[170.26702089157018,
   45.80152671755725,
   3.3242058545136217,
   array([63.00902115, 87.51228128]),
   0.9406013141740135,
   1.085944139308007,
   -0.5204327621930127,
   20]],
 [[173.44004173282804,
   45.80152671755725,
   3.7442805753795056,
   array([63.24182393, 89.25357412]),
   0.9406013141740135,
   1.085944139308007,
   -0.5204327621930127,
   20]],
 [[176.30038448647593,
   45.80152671755725,
   4.9120135213115645,
   array([63.52402316, 89.14249521]),
   0.9406013141740135,
   1.085944139308007,
   -0.5204327621930127,
   20]],
 [[178.90368307274048,
   45.80152671755725,
   4.225995786613424,
   array([63.47957296, 93.00853271]),
   0.9406013141740135,
   1.085944139308007,
   -0.5204327621930127,
   20]],
 [[182.2644240800318,
   45.80152671755725,
   4.8855298279768675,
   array([64.50

In [7]:
import pickle
cell_timeseries_properties_file = open('cell_timeseries_properties.p', 'wb')
pickle.dump(cell_timeseries_properties, cell_timeseries_properties_file)
cell_timeseries_properties_file.close()

main_segments_file = open('main_segments.p', 'wb')
pickle.dump(main_segments, main_segments_file)
main_segments_file.close()